In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import requests
from zipfile import ZipFile

In [16]:
filename = 'adult22'
url = f'https://ftp.cdc.gov/pub/Health_Statistics/NCHS/Datasets/NHIS/2022/{filename}csv.zip'

response = requests.get(url)

with open('temp.zip', 'wb') as file:
    file.write(response.content)

zf = ZipFile('temp.zip')
zf.extractall()
zf.close()

nhis = pd.read_csv(f'{filename}.csv')

os.remove('temp.zip')
os.remove('readme.txt')
os.remove(f'{filename}.csv')

print(f'Shape - {nhis.shape}\n-----\n')

nhis.head()

Shape - (27651, 637)
-----



,URBRRL,RATCAT_A,INCTCFLG_A,IMPINCFLG_A,SHOTTYPE1_A,CEVOTELC_A,CEMMETNG_A,CEVOLUN2_A,CEVOLUN1_A,HITTEST_A,...,PROXYREL_A,PROXY_A,AVAIL_A,HHSTAT_A,INTV_MON,RECTYPE,IMPNUM_A,WTFA_A,HHX,POVRATTC_A
0,2,7,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,1,1,10,1,4548.583,H059086,1.92
1,4,14,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,1,1,10,1,7087.431,H054049,10.30
2,4,14,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,1,1,10,1,8125.516,H055201,9.36
3,4,11,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,1,1,10,1,7837.390,H044893,3.66
4,1,2,0,1,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,1.0,3,1,1,10,1,10234.356,H043149,0.69


In [20]:
for col in nhis.columns:
    if 'FDS' in col:
        print(col)

FDSCAT4_A
FDSCAT3_A
FDSNEDAYS_A
FDSNOTEAT_A
FDSWEIGHT_A
FDSHUNGRY_A
FDSLESS_A
FDSSKIPDYS_A
FDSSKIP_A
FDSBALANCE_A
FDSLAST_A
FDSRUNOUT_A


In [21]:
# filename = 'IND2021ER.txt'
# url = 'https://simba.isr.umich.edu/Zips/GetFileCDS.aspx?file=1053&mainurl=Y'

# response = requests.get(url)

# with open('temp.zip', 'wb') as file:
#     file.write(response.content)

# zf = ZipFile('temp.zip')
# zf.extract(filename)
# zf.close()

# psid = pd.read_csv(filename, sep='|')

# os.remove('temp.zip')
# os.remove(filename)

# print(f'Shape - {nhis.shape}\n-----\n')

# psid.head()

BadZipFile: File is not a zip file